### CAUTION

**This notebook is not ready for teaching. It's more of an exploration of the topic.**

# Forecasting time series

Start here >> [Pandas_for_time_series](Pandas_for_time_series.ipynb)

In that notebook we build a dataframe from Norwegian production data. At the end, we saved it so that we can read it again here.

In [ ]:
import pandas as pd

df = pd.read_csv('npd.csv')

df = df.set_index('ds')

In [ ]:
df.head()

Let's try to predict water cut.

We'll make a new dataframe just for Troll:

In [ ]:
dt = df[df.field=='TROLL'].copy()
dt = dt.drop('field', axis=1)

In [ ]:
dt.head()

## ARIMA

[Autoregressive integrated moving average](https://en.wikipedia.org/wiki/Autoregressive_integrated_moving_average)

We'll instantiate the model with `order=(5, 1, 0)`. This sets the lag value to 5 for autoregression, uses a difference order of 1 to make the time series stationary, and uses a moving average model of 0.

In [ ]:
dt.loc['1996':, 'other'].plot()

In [ ]:
d_train = dt.loc['1996':'2014', 'other']
d_test = dt.loc['2014':, 'other']

In [ ]:
plt.plot(d_train.index, d_train.values)
plt.plot(d_test.index, d_test.values)
plt.xticks([])
plt.show()

In [ ]:
from pandas.plotting import autocorrelation_plot
  
autocorrelation_plot(d_train)
plt.xlim(0, 30)
plt.show()

In [ ]:
from statsmodels.tsa.arima_model import ARIMA
from matplotlib import pyplot
 
model = ARIMA(dt.other, order=(4, 1, 0))
model_fit = model.fit(disp=0)
model_fit.summary()

Let's look at the residuals.

In [ ]:
import matplotlib.pyplot as plt

residuals = pd.DataFrame(model_fit.resid)

fig, axs = plt.subplots(figsize=(15, 3), ncols=2)
residuals.plot(ax=axs[0])
residuals.plot(ax=axs[1], kind='kde')
plt.show()

In [ ]:
residuals.describe()

Let's try auto-ARIMA.

Now following this >> https://medium.com/@josemarcialportilla/using-python-and-auto-arima-to-forecast-seasonal-time-series-90877adff03c

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

result = seasonal_decompose(d_train, model='additive', freq=1)

result.plot()

## Forecasting with Facebook prophet

The Prophet API requires a column called `'ds'` for the dates, and another called `'y'` for the thing we want to predict.

In [ ]:
dt['y'] = dt['other']

In [ ]:
dt.head()

In [ ]:
dt.plot()

In [ ]:
d_train = dt.loc['1996':'2014'].reset_index()
d_test = dt.loc['2014':].reset_index()

In [ ]:
from fbprophet import Prophet

model = Prophet()

model.fit(d_train)

In [ ]:
future = model.make_future_dataframe(periods=240, freq='M')

In [ ]:
forecast = model.predict(future)

In [ ]:
_ = model.plot(forecast)

In [ ]:
fig = model.plot_components(forecast)

## Check validation data

In [ ]:
forecast = model.predict(d_test)

In [ ]:
_ = model.plot(forecast)

In [ ]:
from fbprophet.diagnostics import cross_validation
df_cv = cross_validation(model, initial='3600 days', period='90 days', horizon='180 days')
df_cv.head()

In [ ]:
from fbprophet.plot import plot_cross_validation_metric
fig = plot_cross_validation_metric(df_cv, metric='rmse')

## Logistic growth (with constraints)

Let's look at water:

In [ ]:
d_train['y'] = d_train['water']

model = Prophet()

model.fit(d_train)

future = model.make_future_dataframe(periods=240, freq='M')

forecast = model.predict(future)

fig = model.plot(forecast)

The model has some predictions going below 0, which is not possible. So let's introduce a constraint.

In [ ]:
d_train['cap'] = 3
d_train['floor'] = 0

model = Prophet(growth='logistic')

model.fit(d_train)

In [ ]:
future = model.make_future_dataframe(periods=240, freq='M')
future['cap'] = 3
future['floor'] = 0

forecast = model.predict(future)
fig = model.plot(forecast)

## Controlling the seasonality

The variance is a bit too high; there are ways to control this.

This cell is a little slower to run than the others...

In [ ]:
model = Prophet(growth='logistic', seasonality_mode='multiplicative', mcmc_samples=200)

model.fit(d_train)

future = model.make_future_dataframe(periods=240, freq='M')
future['cap'] = 4
future['floor'] = 0

forecast = model.predict(future)

fig = model.plot(forecast)